
# Vardef demo

Denne demonstrasjonen viser hvordan du kan bruke Vardef-modulen i Dapla toolbelt metadata for å opprette, forvalte og hente ut variabeldefinisjoner. 

Vi anbefaler å bruke Vardef-modeller for å opprette og redigere variabeldefinisjoner.

Nyttige modeller:

- Draft
- Owner
- Contact
- UpdateDraft
- Patch
- ValidityPeriod

Les om alternativ input i seksjonen [Alternativer til Model input](#alternativer-til-model-input)

In [ ]:
from datetime import date
from pprint import pprint

from dapla_metadata.variable_definitions import Vardef
from dapla_metadata.variable_definitions import models



## Drafts

### Opprette et utkast

Alle variabeldefinisjoner får status `DRAFT` ved opprettelse slik at innholdet kan kvalitetsikres før man publiserer.

Det er kun i `DRAFT` at man kan endre `short_name`og det er kun i `DRAFT` at man kan slette en lagret variabeldefinisjon. 

Kan gi en `409 CONFLICT` hvis short_name allerede eksisterer, da kortnavnet må være unikt.

Den som oppretter variabeldefinisjonen blir lagret som `Owner` (teamet og gruppen man representerer).

Alle operasjoner for lagrede variabeldefinisjoner må utføres på en variabel.


In [ ]:
draft = models.Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_shortname",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definisjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

my_draft = Vardef.create_draft(draft)

# Etter opprettelse blir din bruker lagret som `createdBy`
print(my_draft.created_by)


### Eksempel `exception` ved opprettelse av utkast

Vardef-modulen er designet for å hjelpe brukeren med å forvalte variabeldefinisjoner og kvalitetsikre at alle variabeldefinisjoner oppfyller nødvendige krav og regler.

I mange tilfeller blir `VardefClientException` returnert til brukeren med en tydelig feilmelding. Meldingen vil komme på bunnen av `Traceback`.

Det er nyttig å gjøre seg kjent med de ulike feilmeldingene da de gir viktig informasjon om hva som har gått galt og hvor feilen ligger. 

I eksempelet er det ugyldige Klass-koder som utløser en `BadRequestException` (400).


In [ ]:
invalid_draft = models.Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_excpetion",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definosjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "haha",
    ],
    subject_fields=[
        "a",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

Vardef.create_draft(invalid_draft)


#### Nyttig informasjon om variabler

Hvis du trenger å tilordne variabeldefinisjonen din til en variabel igjen og har tilgang til variabeldefinisjon-id:

> my_new_variable = Vardef.get_variable_definition("kWzf65gt") 


Hvis du har mistet oversikten over variabeldefinisjons-id, finnes det eksempler på hvordan du kan hente en spesifikk
variabeldefinisjon i seksjonen [Hente en variabeldefinisjon ved hjelp av kortnavn](#hente-en-variabeldefinisjon-ved-hjelp-av-kortnavn)


In [ ]:
# Vi kan få tilgang til alle feltene direkte fra variabelen.
print("Definisjonstekst på bokmål: ")
pprint(my_draft.definition["nb"])

# Eller du kan få tilgang til feltene ved å hente den lagrede variabeldefinisjonen etter ID for å sjekke verdiene.
print("\nVariabeldefinisjon opprettet: ")
print(Vardef.get_variable_definition(my_draft.id).created_at)

# Feltene blir ikke automatisk oppdatert etter endringer. Hvis du ønsker å fortsette å få tilgang til feltene direkte fra variabelen, kan du oppdatere på denne måten etter hver endring:
my_draft = Vardef.get_variable_definition(my_draft.id)



### Rediger et utkast

Denne operasjonen kan kun utføres på variabeldefinisjoner med status DRAFT. Alle operasjoner på lagrede variabeldefinisjoner er kun tilgjengelige for `Owner`.


In [ ]:
# Oppdater kontaktinformasjon
my_contact = models.Contact(
    title={
        "nb": "Seksjon for befolkningsstatistikk",
        "nn": "Seksjon for befolkningsstatistikk",
        "en": "Division for population statistics",
    },
    email="sibby@ssb.no",
)
update_contact = models.UpdateDraft(
    contact=my_contact,
)

my_draft.update_draft(update_draft=update_contact)
my_draft = Vardef.get_variable_definition(my_draft.id)

print(my_draft.contact)

### Eksempel `ValidationError` ved oppdatering av utkast

Ugyldig kortnavn vil returnerer en `ValidationError`.

In [ ]:
invalid_short_name = models.UpdateDraft(
    short_name="_)(45)",
)
my_draft.update_draft(invalid_short_name)


### Migrere en variabeldefinisjon fra Vardok til Vardef

Etter migreringen blir variabelen lagret som `DRAFT` og kan redigeres på samme måte som vist i [Rediger et utkast](#rediger-et-utkast)




In [ ]:
my_vardok_draft = Vardef.migrate_from_vardok("90")
print(my_vardok_draft)

Hvis Vardok mangler gyldig kortnavn vil et unikt kortnavn bli generert slik at variabelen kan migreres. Merk at dette kortnavnet er ikke gyldig for publisering og må endres i `Draft`.

Alle genererte kortnavn begynner med `generert`.

Under er et eksempel på hvordan eier kan få oversikt over alle variabeldefinisjoner med generert kortnavn.


In [ ]:
my_vardok_missing_short_name = Vardef.migrate_from_vardok("123")
my_owner_team = "dapla-felles"
my_team_variables_generert_kortnavn = [{variable.short_name, variable.id} for variable in Vardef.list_variable_definitions() if variable.owner["team"] == my_owner_team and variable.short_name.startswith("generert")]
print(my_team_variables_generert_kortnavn)


Vardok som mangler verdi for `valid_from` vil få generert en default gyldig fra dato `1900-01-01`.

Denne datoen kan publiseres.

Under er et eksempel på hvordan eier kan få oversikt over alle variabeldefinisjoner med generert gyldig fra dato.

In [ ]:
my_vardok_missing_valid_from = Vardef.migrate_from_vardok("100")
my_owner_team = "dapla-felles"
my_team_variables_generert_valid_from = [{variable.valid_from, variable.id} for variable in Vardef.list_variable_definitions() if variable.owner["team"] == my_owner_team and variable.valid_from == date(1900,1,1)]
print(my_team_variables_generert_valid_from)

#### Eksempel på `exception` ved migrering 

In [ ]:
# Id eksisterer ikke
Vardef.migrate_from_vardok("21")

### Slette et utkast

Denne handlingen kan kun utføres på en variabeldefinisjon med status `DRAFT`.

In [ ]:
my_vardok_draft.delete_draft()

## Publisere

Når utkastet har blitt kvalitetssjekket er det på tide å publisere. 

Tilgjengelige statuser for en variabeldefinisjon:

- DRAFT
- PUBLISHED_INTERNAL
- PUBLISHED_EXTERNAL

Vi anbefaler å benytte seg av variabelstatus-modellen for å motvirke skrivefeil.

For eksempel: 

> models.VariableStatus.PUBLISHED_INTERNAL 

Merk at denne prosessen er irreversibel.

Det er ulike regler som gjelder ved publisering internt og eksternt, men merk også her at prosessen er irreversibel; når en variabeldefinisjon er publisert eksternt kan den ikke endres tilbake til å kun være publisert internt.



In [ ]:
update_status = models.UpdateDraft(
    variable_status=models.VariableStatus.PUBLISHED_INTERNAL,
)

my_draft.update_draft(update_status)

## Patch

### Opprette en Patch

Patcher brukes for mindre endringer som ikke krever en ny `ValidityPeriod`.

Eksempler på grunner for å opprette en ny patch:

- Korrektur av en skrivefeil
- Legge til en oversettelse
- Legge til `subject_field`

Oppgi kun de feltene som skal endres. Andre felter vil beholde sine nåværende verdier.

#### Eksempel oppdatere `Owner`

Legge til gruppe som kan forvalte variabeldefinisjonen. 

In [ ]:
from dapla_metadata.variable_definitions.generated.vardef_client.models.owner import (
    Owner,
)
from dapla_metadata.variable_definitions.generated.vardef_client.models.patch import (
    Patch,
)

new_owner = Patch(
    owner=Owner(
    team="dapla-felles",
    groups=[
        "dapla-felles-developers",
        "play-enhjoern-a-developers",
    ],
),
)

my_draft.create_patch(new_owner)

my_draft = Vardef.get_variable_definition(my_draft.id)

print(my_draft.owner)

#### Overføre eierskap til et annet team

Ved for eksempel omorganiseringer kan det være nødvendig å overføre hele eierskapet til et annet team.

Merk: Når teamet du representerer blir helt fjernet som eier vil du ikke ha tilgang til å vedlikeholde variabeldefinisjonen.



In [ ]:
# Opprette en nytt utkast for å demonstrere overføring av eierskap
owner_draft = models.Draft(
    name = {
        "nb": "test navn",
        "nn": "test namn",
        "en": "test name",
    },
    short_name= "test_owner_2",
    definition= {
        "nb": "Skriv en definisjonstekst på norsk bokmål.",
        "nn": "Skriv ein definisjonstekst på nynorsk.",
        "en": "Write a definition text in english.",
    },
    classification_reference="91",
    unit_types=[
        "01",
    ],
    subject_fields=[
        "al",
    ],
    contains_special_categories_of_personal_data=False,
    measurement_type=None,
    valid_from=date(2024,12,18),
    external_reference_uri=None,
    comment=None,
    related_variable_definition_uris=None,
    contact=None,
)

transfer_owner_draft = Vardef.create_draft(owner_draft)

transfer_owner_draft.update_draft(models.UpdateDraft(variable_status=models.VariableStatus.PUBLISHED_INTERNAL))

replace_owner = Patch(
    owner=Owner(
    team="play-enhjoern-a",
    groups=[
        "play-enhjoern-a-developers",
    ],
),
)

# Oppdatere 'Owner'
transfer_owner_draft.create_patch(replace_owner)

#####  Forsøk på å opprette en ny patch etter overføring av eierskap

Vil føre til en `ForbiddenException`(403)

In [ ]:
transfer_owner_draft.create_patch(Patch(unit_types=["01"]))

### Liste patcher for en variabeldefinisjon

In [ ]:
patches = my_draft.list_patches()

print(f"Patcher for id {my_draft.id}:\n")
for patch in patches:
    print(f"Patch id: {patch.patch_id}\nOpprettet: {patch.created_at}\n")



### Hent en patch

In [ ]:
# Bruk patch_id
print(my_draft.get_patch(2))

## Validity period

For å opprette en ny gyldighetsperiode må inndataene inneholde oppdatert `definition`tekst for alle gjeldende språk og en ny gyldig fra dato.

En ny gyldighetsperiode bør kun opprettes når den grunnleggende definisjonen av variabelen har endret seg.

På denne måten kan den forrige definisjonen bevares for bruk i historiske data.


In [ ]:

valid_validity_period = models.ValidityPeriod(
    definition={
        "nb": "ny definisjon2",
        "nn": "ny definisjon2",
        "en": "new definition2",
    },
    valid_from=date(2040,4,5),
)
my_draft.create_validity_period(valid_validity_period)



#### Eksempel ugyldig `ValidityPeriod`

In [ ]:
invalid_validity_period = models.ValidityPeriod(
    name={
        "nb": "nytt navn",
    },
    valid_from=date(2025,4,5),
)

my_draft.create_validity_period(invalid_validity_period)

### Liste gyldighetsperioder for en variabeldefinisjon

In [ ]:
validity_periods = my_draft.list_validity_periods()

print(f"Gyldighetsperioder for id {my_draft.id}:\n")
for period in validity_periods:
    print(f"Patch id: {period.patch_id}\nGyldig fra: {period.valid_from} til {period.valid_until}\nDefinisjon: {period.definition}\n")



## Liste alle variabeldefinisjoner

Nyeste versjon av alle variabeldefinisjoner hentes ut.



In [ ]:

variable_definitions = Vardef.list_variable_definitions()

for variable in variable_definitions:
    print(f"Id: {variable.id}\nNavn: {variable.name}\nKortnavn: {variable.short_name}\nDefinisjon: {variable.definition}\nEier: {variable.owner}\n")



### Filtrere på dato

Nyttig hvis man ønsker tilgang til en spesifikk gyldighetsperiode.


In [ ]:

desired_validity = date(2030,2,24)
filtered_variable_definitions = Vardef.list_variable_definitions(desired_validity)

print(f"Valid at {desired_validity}:\n")
for variable in filtered_variable_definitions:
    print(f"Id: {variable.id}\nName: {variable.name}\nShort name: {variable.short_name}\nDefinition: {variable.definition}\nOwner: {variable.owner}\nValid: {variable.valid_from} to {variable.valid_until}\n")

### Skrive egne filteroperasjoner

Vardef støtter ikke annen filtrering enn dato, men det er enkelt å filtrere på ønsket attributt og tilpasse resultatet etter behov.

I de påfølgende eksemplene baserer vi oss på variabelen `variable_definitions`som inneholder en liste over alle lagrede variabeldefinisjoner.

Under følger noen eksempler der man filtrere på eierteam og status.




In [ ]:
owner_team = "dapla-felles"
print(f"\nFiltrer etter eierteam {owner_team}: ")
my_team_variables = [variable for variable in variable_definitions if variable.owner["team"] == owner_team]

# Eksempel formattert print som kun printer verdier på norsk bokmål. Hvis det mangler verdi på valgt språk blir feltet tomt.
for variable in my_team_variables:
    print(f"Id: {variable.id}\nName: {variable.name['nb']}\nShort name: {variable.short_name}\nDefinition: {variable.definition['nb']}\nOwner: {variable.owner['team']}\n")


print("\nFiltrer etter status `DRAFT`: ")
draft_variables = [variable for variable in variable_definitions if variable.variable_status == models.VariableStatus.DRAFT]
print(draft_variables)

print("\nFiltrer etter status `PUBLISHED INTERNAL`: ")
published_intern_variables = [variable for variable in variable_definitions if variable.variable_status == models.VariableStatus.PUBLISHED_INTERNAL]
print(published_intern_variables)


#### Hent en variabeldefinisjon

Hent en ved å sende inn `id`. 


In [ ]:
print(Vardef.get_variable_definition(my_draft.id))

#### Hente en variabeldefinisjon ved hjelp av kortnavn

Hvis man mister oversikten over `id` er dette et eksempel på hvordan man kan hente ut `id` ved å bruke kortnavn som også er unikt. 

Denne operasjonen er en filtrering på listen `variable_definitions`.

Alle operasjoner som er lov i henhold til status kan utføres på variabelen.

In [ ]:

variable_by_short_name = next(variable for variable in variable_definitions if variable.short_name == "test_shortname")

print(variable_by_short_name.id)

## Alternativer til `Model` input

Hvis man ønsker det er det også mulig å sende inn `dictionary`.

In [ ]:
NEW_DRAFT = {
    "name": {"en": "Country Background", "nb": "Landbakgrunn", "nn": "Landbakgrunn"},
    "short_name": "new_short_name1",
    "definition": {
        "en": "Country background is the person's own, the mother's or possibly the father's country of birth. Persons without an immigrant background always have Norway as country background. In cases where the parents have different countries of birth the mother's country of birth is chosen. If neither the person nor the parents are born abroad, country background is chosen from the first person born abroad in the order mother's mother, mother's father, father's mother, father's father.",
        "nb": "For personer født i utlandet, er dette (med noen få unntak) eget fødeland. For personer født i Norge er det foreldrenes fødeland. I de tilfeller der foreldrene har ulikt fødeland, er det morens fødeland som blir valgt. Hvis ikke personen selv eller noen av foreldrene er utenlandsfødt, hentes landbakgrunn fra de første utenlandsfødte en treffer på i rekkefølgen mormor, morfar, farmor eller farfar.",
        "nn": "For personar fødd i utlandet, er dette (med nokre få unntak) eige fødeland. For personar fødd i Noreg er det fødelandet til foreldra. I dei tilfella der foreldra har ulikt fødeland, er det fødelandet til mora som blir valt. Viss ikkje personen sjølv eller nokon av foreldra er utenlandsfødt, blir henta landsbakgrunn frå dei første utenlandsfødte ein treffar på i rekkjefølgja mormor, morfar, farmor eller farfar.",
    },
    "classification_reference": "91",
    "unit_types": ["01", "02"],
    "subject_fields": ["he04"],
    "contains_special_categories_of_personal_data": True,
    "measurement_type": None,
    "valid_from": "2003-01-01",
    "external_reference_uri": "https://www.ssb.no/a/metadata/conceptvariable/vardok/1919/nb",
    "comment": {
        "nb": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "nn": "Fra og med 1.1.2003 ble definisjon endret til også å trekke inn besteforeldrenes fødeland.",
        "en": "As of 1 January 2003, the definition was changed to also include the grandparents' country of birth.",
    },
    "related_variable_definition_uris": ["https://example.com/"],
    "contact": {
        "title": {
            "en": "Division for population statistics",
            "nb": "Seksjon for befolkningsstatistikk",
            "nn": "Seksjon for befolkningsstatistikk",
        },
        "email": "s320@ssb.no",
    },
}

